<pre>
1. Download all the data in this folder https://drive.google.com/open?id=1Z4TyI7FcFVEx8qdl4jO9qxvxaqLSqoEu. it contains two file both images and labels. The label file list the images and their categories in the following format:
            <b>path/to/the/image.tif,category</b>
            
    where the categories are numbered 0 to 15, in the following order:

    <b>0 letter
    1 form
    2 email
    3 handwritten
    4 advertisement
    5 scientific report
    6 scientific publication
    7 specification
    8 file folder
    9 news article
    10 budget
    11 invoice
    12 presentation
    13 questionnaire
    14 resume
    15 memo</b>
    
2. On this image data, you have to train 3 types of models as given below. You have to split the data into Train and Validation data.

3. Try not to load all the images into memory, use the gernarators that we have given the reference notebooks to load the batch of images only during the train data.
or you can use this method also
<a href='https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1'>https://medium.com/@vijayabhaskar96/tutorial-on-keras-imagedatagenerator-with-flow-from-dataframe-8bd5776e45c1</a>

<a href='https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c'>https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c</a>


4. You are free to choose Learning rate, optimizer, loss function, image augmentation, any hyperparameters. but you have to use the same architechture what we are asking below. 

5. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

Note: fit_genarator() method will have problems with the tensorboard histograms, try to debug it, if you could not do use histgrams=0 i.e don't include histograms, check the documentation of tensorboard for more information. 

6. You can check about Transfer Learning in this link - <a href='https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html'>https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html</a>
</pre>

### Model-1

<pre>
1. Use <a href='https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16'>VGG-16</a> pretrained network without Fully Connected layers and initilize all the weights with Imagenet trained weights. 
2. After VGG-16 network without FC layers, add a new Conv block ( 1 Conv layer and 1 Maxpooling ), 2 FC layers and a output layer to classify 16 classes. You are free to choose any hyperparameters/parameters of conv block, FC layers, output layer. 
3. Final architecture will be <b>INPUT --> VGG-16 without Top layers(FC) --> Conv Layer --> Maxpool Layer --> 2 FC layers --> Output Layer</b>
4. Train only new Conv block, FC layers, output layer. Don't train the VGG-16 network. 

</pre>

In [1]:
#!pip install keras

In [3]:
#!pip uninstall keras-preprocessing

In [2]:
#!pip install git+https://github.com/keras-team/keras-preprocessing.git

In [1]:
from keras_preprocessing.image import ImageDataGenerator

In [3]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [4]:
total_labels = pd.read_csv('rvl-cdip/labels_final.csv')

In [5]:
total_labels.head()

,path,label
0,imagesv/v/o/h/voh71d00/509132755+-2755.tif,3
1,imagesl/l/x/t/lxt19d00/502213303.tif,3
2,imagesx/x/e/d/xed05a00/2075325674.tif,2
3,imageso/o/j/b/ojb60d00/517511301+-1301.tif,3
4,imagesq/q/z/k/qzk17e00/2031320195.tif,7


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers 
import re

In [7]:
#https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
dataframe=total_labels,
directory="rvl-cdip/data_final",
x_col="path",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(224,224))

valid_generator=datagen.flow_from_dataframe(
dataframe=total_labels,
directory="rvl-cdip/data_final",
x_col="path",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(224,224))

Found 36000 validated image filenames.
Found 12000 validated image filenames.


In [8]:
print('len of train labels: ',len(train_generator.labels))
print('len of valid labels: ',len(valid_generator.labels))

len of train labels:  36000
len of valid labels:  12000


In [9]:
##Checking time taken to load images. 
import time
start = time.time()
total_batches = 0

batches = 0
per_batch = 32
for x_batch, y_batch in train_generator:
    batches += 1
    if batches >= 36000/per_batch:
        total_batches = total_batches + batches
        break 
end = time.time()
duration = end-start
print("{} batches: {} s".format(total_batches, duration))
print("{:0.5f} Images/s".format(per_batch*total_batches/duration))

1125 batches: 249.88824820518494 s
144.06440 Images/s


In [10]:
##Checking time taken to load images. 
import time
start = time.time()
total_batches = 0

batches = 0
per_batch = 32
for x_batch, y_batch in valid_generator:
    batches += 1
    if batches >= 12000/per_batch:
        total_batches = total_batches + batches
        break 
end = time.time()
duration = end-start
print("{} batches: {} s".format(total_batches, duration))
print("{:0.5f} Images/s".format(per_batch*total_batches/duration))

375 batches: 86.22054624557495 s
139.17796 Images/s


In [11]:
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten,BatchNormalization
from tensorflow.keras.models import Model
import random as rn
from tensorflow.keras.applications.vgg16 import VGG16

In [87]:
import os
os.environ['PYTHONHASHSEED'] = '0'

tf.keras.backend.clear_session()

np.random.seed(0)
rn.seed(0)

model_1 = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))
model_1.trainable = False
model_1_out = model_1.output 


In [88]:
for i, layer in enumerate(model_1.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [89]:
#Conv Layer
conv1 = Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(seed=0),name='Conv1')(model_1_out)

#MaxPool Layer
mxPool1 = MaxPool2D(pool_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',name='Pool1')(conv1)

#Flatten
flatten = Flatten(data_format='channels_last',name='Flatten')(mxPool1)

#FC layer
FC1 = Dense(units=256,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0),name='FC1')(flatten)

#FC layer
FC2 = Dense(units=64,activation='relu',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0),name='FC2')(FC1)

#output layer
Out = Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=0),name='Output')(FC2)

#Creating a model
model = Model(inputs=model_1.input,outputs=Out)

In [90]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [91]:
def lr_update(epoch,lr):
    if epoch%5 == 0 and lr>1e-4:
        return lr - (0.1*lr)
    return lr

In [92]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TerminateOnNaN
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
import datetime

log_dir="logs1/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True)

path="checkpts/model1_save.hdf5"
model_check = ModelCheckpoint(filepath=path, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

terminate = TerminateOnNaN()
learning_rate = LearningRateScheduler(lr_update,verbose=1)

early_stop = EarlyStopping(monitor="val_accuracy",patience=5,mode='auto')

call_back=[tensorboard,model_check,terminate,learning_rate,early_stop]

In [93]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [94]:
model.fit(train_generator,steps_per_epoch=1125,epochs=30,validation_data=valid_generator,validation_steps=375,callbacks=call_back)

Epoch 1/30

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
1125/1125 [==============================] - 789s 700ms/step - loss: 1.4132 - accuracy: 0.5672 - val_loss: 1.1250 - val_accuracy: 0.6654

Epoch 00001: val_accuracy improved from -inf to 0.66542, saving model to checkpts\model1_save.hdf5
Epoch 2/30

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
1125/1125 [==============================] - 1328s 1s/step - loss: 0.9873 - accuracy: 0.6936 - val_loss: 0.9938 - val_accuracy: 0.6998

Epoch 00002: val_accuracy improved from 0.66542 to 0.69975, saving model to checkpts\model1_save.hdf5
Epoch 3/30

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
1125/1125 [==============================] - 1470s 1s/step - loss: 0.8359 - accuracy: 0.7433 - val_loss: 0.9394 - val_accuracy: 0.7198

Epoch 00003: val_accuracy improved from 0.69975 to 0.71983, saving model to checkpts\model1_save.hdf5

In [1]:
%load_ext tensorboard

In [5]:
%tensorboard --logdir logs1/fit/

Reusing TensorBoard on port 6006 (pid 10520), started 1 day, 16:11:04 ago. (Use '!kill 10520' to kill it.)

<img src="model1/accuracy.png">

<img src="model1/graph.png">
<img src="model1/distributions.png">
<img src="model1/histograms.png">
<img src="model1/timeseries.png">

### Model-2

<pre>
1. Use <a href='https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG16'>VGG-16</a> pretrained network without Fully Connected layers and initilize all the weights with Imagenet trained weights.
2. After VGG-16 network without FC layers, don't use FC layers, use conv layers only as Fully connected layer. any FC layer can be converted to a CONV layer. This conversion will reduce the No of Trainable parameters in FC layers. For example, an FC layer with K=4096 that is looking at some input volume of size 7×7×512 can be equivalently expressed as a CONV layer with F=7,P=0,S=1,K=4096. In other words, we are setting the filter size to be exactly the size of the input volume, and hence the output will simply be 1×1×4096 since only a single depth column “fits” across the input volume, giving identical result as the initial FC layer. You can refer <a href='http://cs231n.github.io/convolutional-networks/#convert'>this</a> link to better understanding of using Conv layer in place of fully connected layers.
3. Final architecture will be VGG-16 without FC layers(without top), 2 Conv layers identical to FC layers, 1 output layer for 16 class classification. <b>INPUT --> VGG-16 without Top layers(FC) --> 2 Conv Layers identical to FC --> Output Layer</b>
3. Train only last 2 Conv layers identical to FC layers, 1 output layer. Don't train the VGG-16 network. 
</pre>

In [25]:
model2 = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))
model2.trainable = False
model2_out = model2.output 

In [26]:
for i, layer in enumerate(model2.layers):
    print(i, layer.name, layer.trainable)

0 input_3 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 False
14 block4_pool False
15 block5_conv1 False
16 block5_conv2 False
17 block5_conv3 False
18 block5_pool False


In [42]:
conv1 = Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(),name='Conv1')(model2_out)


conv2 = Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(),name='Conv2')(conv1)


flatten = Flatten(data_format='channels_last',name='Flatten')(conv2)


Out2 = Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(seed=3),name='Output')(flatten)

#model2
model2_final = Model(inputs=model2.input,outputs=Out2)

In [43]:
model2_final.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [44]:
log_dir="logs2/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True)

path="checkpts/model2_save.hdf5"
model_check = ModelCheckpoint(filepath=path, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

terminate = TerminateOnNaN()
learning_rate = LearningRateScheduler(lr_update,verbose=1)

early_stop = EarlyStopping(monitor="val_accuracy",patience=5,mode='auto')

call_back=[tensorboard,model_check,terminate,learning_rate,early_stop]

In [45]:
model2_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [46]:
model2_final.fit(train_generator,steps_per_epoch=1125,epochs=50,validation_data=valid_generator,validation_steps=375,callbacks=call_back)

Epoch 1/50

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
1125/1125 [==============================] - 1022s 908ms/step - loss: 1.4010 - accuracy: 0.5738 - val_loss: 1.1482 - val_accuracy: 0.6567

Epoch 00001: val_accuracy improved from -inf to 0.65667, saving model to checkpts\model2_save.hdf5
Epoch 2/50

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
1125/1125 [==============================] - 462s 410ms/step - loss: 1.0181 - accuracy: 0.6902 - val_loss: 1.0177 - val_accuracy: 0.6988

Epoch 00002: val_accuracy improved from 0.65667 to 0.69883, saving model to checkpts\model2_save.hdf5
Epoch 3/50

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0009000000427477062.
1125/1125 [==============================] - 400s 356ms/step - loss: 0.8724 - accuracy: 0.7320 - val_loss: 1.0363 - val_accuracy: 0.6954

Epoch 00003: val_accuracy did not improve from 0.69883
Epoch 4/50

Epoch 00004: LearningRateSche

In [1]:
%load_ext tensorboard

In [4]:
%tensorboard --logdir logs2/fit/

Reusing TensorBoard on port 6006 (pid 11020), started 1 day, 1:06:11 ago. (Use '!kill 11020' to kill it.)

<img src="model2/accuracy.png">
<img src="model2/graph.png">
<img src="model2/distributions.png">
<img src="model2/histograms.png">
<img src="model2/timeseries.png">

### Model-3

<pre>
1. Use same network as Model-2 '<b>INPUT --> VGG-16 without Top layers(FC) --> 2 Conv Layers identical to FC --> Output Layer</b>' and train only Last 6 Layers of VGG-16 network, 2 Conv layers identical to FC layers, 1 output layer.
</pre>

In [140]:
import os
import numpy as np
os.environ['PYTHONHASHSEED'] = '0'


##https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
## Have to clear the session. If you are not clearing, Graph will create again and again and graph size will increses. 
## Varibles will also set to some value from before session
tf.keras.backend.clear_session()

## Set the random seed values to regenerate the model.
np.random.seed(0)
rn.seed(0)

In [141]:
lst5=[1,4,2,5,2,7,3,6]
lst5[:4]

[1, 4, 2, 5]

In [142]:
model3 = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))
#base_model.trainable = False
for layer in model3.layers[0:-6]:
    layer.trainable = False
model3_out = model3.output 

In [143]:
for i, layer in enumerate(model3.layers):
    print(i, layer.name, layer.trainable)

0 input_1 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_pool False
11 block4_conv1 False
12 block4_conv2 False
13 block4_conv3 True
14 block4_pool True
15 block5_conv1 True
16 block5_conv2 True
17 block5_conv3 True
18 block5_pool True


In [144]:
#conv
conv1 = Conv2D(filters=512,kernel_size=(5,5),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(),name='Conv1')(model3_out)

#conv
conv2 = Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='valid',data_format='channels_last',
              activation='relu',kernel_initializer=tf.keras.initializers.he_normal(),name='Conv2')(conv1)

#flatten
flatten = Flatten(data_format='channels_last',name='Flatten')(conv2)

#out layer
Out = Dense(units=16,activation='softmax',kernel_initializer=tf.keras.initializers.glorot_normal(),name='Output')(flatten)

#model
model3_final = Model(inputs=model3.input,outputs=Out)

In [145]:
model3_final.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [146]:
def lr_update(epoch,lr):
    if epoch<=10:
        return lr 
    else:
        if epoch%5 == 0 and lr>1e-4:
            return lr - (0.1*lr)
    return lr

In [147]:
log_dir="logs3_fi/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True)

path="checkpts/model3_save.hdf5"
model_check = ModelCheckpoint(filepath=path, monitor='val_accuracy',  verbose=1, save_best_only=True, mode='auto')

terminate = TerminateOnNaN()
learning_rate = LearningRateScheduler(lr_update,verbose=1)

early_stop = EarlyStopping(monitor="val_accuracy",patience=3,mode='auto')

call_back=[tensorboard,model_check,terminate,learning_rate,early_stop]

In [148]:
model3_final.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [149]:
model3_final.fit(train_generator,steps_per_epoch=1125,epochs=15,validation_data=valid_generator,validation_steps=375,callbacks=call_back)

Epoch 1/15

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
1125/1125 [==============================] - 538s 477ms/step - loss: 2.7892 - accuracy: 0.0611 - val_loss: 2.7727 - val_accuracy: 0.0634

Epoch 00001: val_accuracy improved from -inf to 0.06342, saving model to checkpts\model3_save.hdf5
Epoch 2/15

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
1125/1125 [==============================] - 543s 483ms/step - loss: 2.7728 - accuracy: 0.0605 - val_loss: 2.7729 - val_accuracy: 0.0635

Epoch 00002: val_accuracy improved from 0.06342 to 0.06350, saving model to checkpts\model3_save.hdf5
Epoch 3/15

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
1125/1125 [==============================] - 544s 484ms/step - loss: 2.7728 - accuracy: 0.0618 - val_loss: 2.7730 - val_accuracy: 0.0613

Epoch 00003: val_accuracy did not improve from 0.06350
Epoch 4/15

Epoch 00004: LearningRateSched

In [1]:
%load_ext tensorboard

In [3]:
%tensorboard --logdir logs3_fi/fit/

Reusing TensorBoard on port 6006 (pid 16540), started 2:30:25 ago. (Use '!kill 16540' to kill it.)

<img src="model3/accuracy.png">
<img src="model3/graph.png">
<img src="model3/distributions.png">
<img src="model3/histograms.png">
<img src="model3/timeseries.png">